# 加载云端硬盘，并更改工作目录

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('/content/drive/MyDrive/ColabNotebooks/benchmarking_pytorchvideo_and_mmaction2')

# 测试mmaction2的模型

## Install

In [3]:
!pip install git+https://github.com/open-mmlab/mim.git
!mim install mmaction2

  Cloning https://github.com/open-mmlab/mim.git to /tmp/pip-req-build-du8okeev
  Running command git clone -q https://github.com/open-mmlab/mim.git /tmp/pip-req-build-du8okeev
mmaction2==0.19.0 existed.


In [4]:
# 验证安装
import torch
from mmaction.apis import init_recognizer, inference_recognizer

# 云端硬盘中的路径
# config_file为mmaction2的下载地址
config_file = '/content/drive/MyDrive/ColabNotebooks/benchmarking_pytorchvideo_and_mmaction2/mmaction2/configs/recognition/tsn/tsn_r50_video_inference_1x1x3_100e_kinetics400_rgb.py'
device = 'cuda:0' # 或 'cpu'
device = torch.device(device)

model = init_recognizer(config_file, device=device)
# 进行演示视频的推理
inference_recognizer(model, '/content/drive/MyDrive/ColabNotebooks/benchmarking_pytorchvideo_and_mmaction2/mmaction2/demo/demo.mp4')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[(360, 31.335176),
 (136, 30.411058),
 (86, 30.067778),
 (201, 27.952486),
 (202, 27.931368)]

## Imports

In [5]:
import torch

from mmaction.apis import init_recognizer, inference_recognizer

from benchmark.MMAction2ModelInspector import MMAction2ModelInspector

## Setup

In [6]:
config_file = 'mmaction2/configs/recognition/tsn/tsn_r50_video_inference_1x1x3_100e_kinetics400_rgb.py'
# 从模型库中下载检测点，并把它放到 `checkpoints/` 文件夹下
checkpoint_file = 'mmaction2/checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'

# 指定设备
device = 'cuda:0' # or 'cpu'
device = torch.device(device)

# 测试视频
video = 'mmaction2/demo/demo.mp4'

In [7]:
mmaction2_model_inspector = MMAction2ModelInspector(video, device, config_file, checkpoint_file)

Use load_from_local loader


In [8]:
mmaction2_model_inspector.run_model()

 latency: 3.2862 sec throughput: 0.3043 req/sec
 latency: 3.3227 sec throughput: 0.3010 req/sec
 latency: 3.3136 sec throughput: 0.3018 req/sec
 latency: 3.2980 sec throughput: 0.3032 req/sec
 latency: 3.3253 sec throughput: 0.3007 req/sec
 latency: 3.3111 sec throughput: 0.3020 req/sec
 latency: 3.3042 sec throughput: 0.3026 req/sec
 latency: 3.2852 sec throughput: 0.3044 req/sec
 latency: 3.2980 sec throughput: 0.3032 req/sec
 latency: 3.3001 sec throughput: 0.3030 req/sec
 latency: 3.2897 sec throughput: 0.3040 req/sec
 latency: 3.2816 sec throughput: 0.3047 req/sec
 latency: 3.2944 sec throughput: 0.3035 req/sec
 latency: 3.2714 sec throughput: 0.3057 req/sec
 latency: 3.2891 sec throughput: 0.3040 req/sec
 latency: 3.2771 sec throughput: 0.3052 req/sec
 latency: 3.2863 sec throughput: 0.3043 req/sec
 latency: 3.2739 sec throughput: 0.3054 req/sec
 latency: 3.2945 sec throughput: 0.3035 req/sec
 latency: 3.2647 sec throughput: 0.3063 req/sec


total batches: 20, batch_size: 1
total

{'batch_size': 1,
 'completed_time': datetime.datetime(2021, 10, 10, 8, 51, 51, 166072),
 'total_batches': 20,
 'total_latency': 65.87604093551636,
 'total_throughput': 0.30360051569549035}

# 测试PyTorchVideo的模型

## Install

In [9]:
!pip install pytorchvideo

## Imports

In [10]:
import torch
import json
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
)
from typing import Dict

from benchmark.PyTorchVideoModelInspector import PyTorchVideoModelInspector

import torchvision

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The _functional_video module is deprecated. Please use the functional module instead.
  "The _functional_video module is deprecated. Please use the functional module instead."
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/_transforms_video.py:26: UserWarning: The _transforms_video module is deprecated. Please use the transforms module instead.
  "The _transforms_video module is deprecated. Please use the transforms module instead."


## Setup

In [11]:
# Device on which to run the model
# Set to cuda to load on GPU
# Set to cpu to load on CPU
device = "cuda"

# Pick a pretrained model and load the pretrained weights
model_name = "slowfast_r50"

video_path = "data/archery.mp4"

In [12]:
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
alpha = 4

In [13]:
pytorchvideo_model_inspector = PyTorchVideoModelInspector(
            video_path, 
            device,
            model_name,
            side_size, 
            mean, 
            std, 
            crop_size, 
            num_frames, 
            sampling_rate, 
            frames_per_second,  
            alpha
            )

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


## Infer

In [14]:
pytorchvideo_model_inspector.run_model()

 latency: 0.1855 sec throughput: 5.3912 req/sec
 latency: 0.1846 sec throughput: 5.4184 req/sec
 latency: 0.1823 sec throughput: 5.4842 req/sec
 latency: 0.1850 sec throughput: 5.4059 req/sec
 latency: 0.1831 sec throughput: 5.4618 req/sec
 latency: 0.1793 sec throughput: 5.5772 req/sec
 latency: 0.1811 sec throughput: 5.5219 req/sec
 latency: 0.1830 sec throughput: 5.4649 req/sec
 latency: 0.1826 sec throughput: 5.4772 req/sec
 latency: 0.1821 sec throughput: 5.4918 req/sec
 latency: 0.1843 sec throughput: 5.4247 req/sec
 latency: 0.1807 sec throughput: 5.5348 req/sec
 latency: 0.1814 sec throughput: 5.5120 req/sec
 latency: 0.1803 sec throughput: 5.5453 req/sec
 latency: 0.1818 sec throughput: 5.4999 req/sec
 latency: 0.1807 sec throughput: 5.5348 req/sec
 latency: 0.1853 sec throughput: 5.3957 req/sec
 latency: 0.1833 sec throughput: 5.4542 req/sec
 latency: 0.1848 sec throughput: 5.4103 req/sec
 latency: 0.1804 sec throughput: 5.5418 req/sec


total batches: 20, batch_size: 1
total

{'batch_size': 1,
 'completed_time': datetime.datetime(2021, 10, 10, 8, 52, 1, 362527),
 'total_batches': 20,
 'total_latency': 3.658512592315674,
 'total_throughput': 5.46670251785054}